In [1]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
import time
from datetime import datetime, timedelta
import calendar
import datetime as dt
from functools import reduce
import json
import os
from collections import Counter

In [2]:
class FinRatio():
    m_mapping = {'上市':'sii','上櫃':'otc'}
    base_url = 'https://mops.twse.com.tw/mops/web/'
    after_ifrs_mapping = {
        '營益分析':base_url+'t163sb06',
        '財務結構分析':base_url+'t51sb02'
    }
    before_ifrs_mapping = {
        '營益分析':base_url+'t51sb06',
        '財務結構分析':base_url+'ajax_t51sb02'
    }

    def __init__(self, market, year, season, table):
        self.year =  year if year < 1000 else year-1911
        self.season = '0'+str(season) if type(season)==int else season
        self.market = self.m_mapping[market] if market in self.m_mapping else None
        self.table = table
        table_mapping = self.after_ifrs_mapping if year>=102 else self.before_ifrs_mapping
        self.url = table_mapping[self.table] if self.table in table_mapping else None

    def get_raw(self):
        form = {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':self.market,
            'year':str(self.year),
            'season':self.season
        }
        form['ifrs'] = 'Y' if self.year >= 102 else 'N'            
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        r = requests.post(self.url, form, headers=headers)
        r.encoding = 'utf8'
        dfs = pd.read_html(StringIO(r.text))
        self.raw = dfs

    def parse(self):
        if self.table=='營益分析':
            dfs = [i for i in self.raw if i.shape[0]>5]
            dfs = [i for i in dfs if i.shape[1]>5]
            self.data = None
            if len(dfs) != 0:
                data = dfs[0]
                data.columns = ['公司代號','公司名稱','營業收入(百萬元)','毛利率(%)','營業利益率(%)','稅前純益率(%)','稅後純益率(%)']
                data = data[data['公司代號']!='公司代號']
                data = data.reset_index().drop('index', axis=1)
                data['財報年度'] = f"{self.year+1911}-{self.season}"
                self.data = data

        elif self.table=='財務結構分析':
            dfs = [i for i in self.raw if i.shape[0]>5]
            dfs = [i for i in dfs if i.shape[1]>5]
            self.data = None
            if len(dfs) != 0:
                data = dfs[0]
                data.columns = data.columns.get_level_values(1)
                data = data[data['公司代號']!='公司代號']
                data = data.reset_index().drop('index', axis=1)
                data.rename(columns={'股東權益報酬率(%)':'權益報酬率(%)'
                                    ,'不動產、廠房及設備週轉率(次)':'固定資產週轉率(次)'
                                    ,'負債佔資產比率(%)':'負債比率(%)'
                                    ,'公司簡稱':'公司名稱'}, inplace=True)
                data['財報年度'] = self.year+1911 #yyyy
                # if self.smp==True :
                #     clms = list(data.columns)
                #     clms = [i for i in clms if i not in \
                #            ['平均收現日數','平均售貨日數','平均銷貨日數'
                #             ,'長期資金佔不動產、廠房及設備比率(%)','純益率(%)','長期資金佔固定資產比率(%)'
                #             ,'應收款項收現日數','稅前純益佔實收資本比率(%)','營業利益佔實收資本比率(%)']]
                #     data = data[clms]
                self.data = data

In [45]:
FinRatio.after_ifrs_mapping

{'營益分析': 'https://mops.twse.com.tw/mops/web/t163sb06',
 '財務結構分析': 'https://mops.twse.com.tw/mops/web/t51sb02'}

In [3]:
fin_ratio = FinRatio('上市', 104, 4, '營益分析')

In [7]:
list(range(1,12))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [4]:
FRs = []
for i in range(1,4):
    fin_ratio = FinRatio('上市', 104, i, '營益分析')
    fin_ratio.get_raw()
    fin_ratio.parse()
    if fin_ratio.data is not None:
        FRs.append(fin_ratio.data)
    zzz = np.random.randint(5,10)
    time.sleep(zzz)

In [6]:
FRs = pd.concat(FRs, axis=0)

In [8]:
FRs.set_index(pd.to_datetime(FRs['財報年度']))

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%),財報年度
財報年度,,,,,,,,
2015-01-01,1101,台泥,21440.14,13.34,7.90,6.86,4.55,2015-01
2015-01-01,1102,亞泥,15362.53,10.62,6.75,12.01,10.04,2015-01
2015-01-01,1103,嘉泥,671.36,-11.27,-27.67,-31.21,-27.30,2015-01
2015-01-01,1104,環泥,1327.43,8.30,1.85,20.32,19.73,2015-01
2015-01-01,1108,幸福,1290.90,20.88,15.37,14.48,12.01,2015-01
...,...,...,...,...,...,...,...,...
2015-03-01,9944,新麗,2981.20,23.06,5.96,6.37,3.64,2015-03
2015-03-01,9945,潤泰新,8787.50,27.74,13.40,81.40,79.97,2015-03
2015-03-01,9946,三發地產,3247.78,29.56,21.00,21.14,14.24,2015-03


In [27]:
FRs[FRs['公司代號']=='1101']

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%),財報年度,季
0,1101,台泥,21440.14,13.34,7.90,6.86,4.55,2015,01
0,1101,台泥,46434.33,14.60,9.93,10.15,7.97,2015,02
0,1101,台泥,69118.30,14.59,9.56,8.51,6.79,2015,03
0,1101,台泥,93679.08,15.51,10.33,9.25,7.39,2015,04


In [19]:
fin_ratio.data

In [4]:
fin_ratio.get_raw()

In [5]:
fin_ratio.parse()

In [6]:
fin_ratio.data

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%),財報年度,季
0,1101,台泥,93679.08,15.51,10.33,9.25,7.39,2015,04
1,1102,亞泥,66287.48,10.65,6.09,10.28,7.44,2015,04
2,1103,嘉泥,2826.52,-3.81,14.47,6.90,7.55,2015,04
3,1104,環泥,5163.75,10.41,3.36,28.50,26.78,2015,04
4,1108,幸福,5021.35,18.05,12.43,12.27,9.22,2015,04
...,...,...,...,...,...,...,...,...,...
887,9944,新麗,3857.00,22.52,4.14,4.06,1.99,2015,04
888,9945,潤泰新,12978.92,28.53,16.81,66.88,65.74,2015,04
889,9946,三發地產,5014.31,29.33,20.48,20.93,14.87,2015,04
890,9955,佳龍,2317.26,-0.60,-5.40,-5.66,-5.49,2015,04


In [16]:
def financial_statement(year, season, type='綜合損益彙總表'):

    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':'103',
        'season':'02',
    })

    r.encoding = 'utf8'
    dfs = pd.read_html(r.text, header=None)

    return pd.concat(dfs[1:], axis=0, sort=False)\
             .set_index(['公司代號'])\
             .apply(lambda s: pd.to_numeric(s, errors='ceorce'))

In [17]:
df = financial_statement(108,'07','營益分析彙總表')

KeyError: "None of ['公司代號'] are in the columns"

In [43]:
 m_mapping = {'上市':'sii','上櫃':'otc'}
base_url = 'https://mops.twse.com.tw/mops/web/'
p_mapping = {
                '營益分析':base_url+'t163sb06'
            ,'財務結構分析':base_url+'t51sb02'}
old_p_mapping = {
                '營益分析':base_url+'t51sb06'
            ,'財務結構分析':base_url+'ajax_t51sb02'}
mkt_type = m_mapping['上市']
url = p_mapping['營益分析']
year = 101
season = '05'
form = {'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':mkt_type,
        'year':str(year),
        'season':season
        }
form['ifrs']='Y' if year>=102 else 'N'            
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}    
r = requests.post(url, form, headers=headers)
r.encoding = 'utf8'
dfs = pd.read_html(StringIO(r.text))

In [45]:
dfs1 = [i for i in dfs if i.shape[0]>5]

In [46]:
dfs1

[]

In [37]:
df.columns = df.columns.get_level_values(1)

In [38]:
df.head()

,公司代號,公司簡稱,負債佔資產比率(%),長期資金佔固定資產比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),應收款項收現日數,存貨週轉率(次),...,總資產週轉率(次),資產報酬率(%),股東權益報酬率(%),營業利益佔實收資本比率(%),稅前純益佔實收資本比率(%),純益率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,13.24,319.70,158.69,142.34,415.58,4.85,75.25,13.64,...,0.21,6.95,7.87,5.57,21.55,32.52,2.09,80.52,109.97,0.67
1,1102,亞泥,31.32,800.62,114.62,95.35,10.72,9.68,37.70,6.28,...,0.10,5.50,7.29,0.78,19.91,53.03,1.93,57.36,85.25,-0.97
2,1103,嘉泥,39.20,1301.91,325.63,313.75,-382.13,9.07,40.24,11.52,...,0.06,-2.58,-4.89,-1.20,-8.78,-56.69,-1.13,31.25,59.58,0.95
3,1104,環泥,13.07,361.77,205.54,165.21,2960.90,4.31,84.68,12.41,...,0.23,4.75,5.46,-0.81,11.81,20.63,1.18,36.15,69.64,-0.65
4,1108,幸福,45.28,212.90,110.67,77.67,4.59,5.43,67.21,7.14,...,0.44,2.86,4.20,1.89,5.04,5.14,0.43,24.98,83.10,3.22


In [31]:
df[df['公司代號']!='公司代號']

,公司代號,公司簡稱,負債佔資產比率(%),長期資金佔不動產、廠房及設備比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),平均收現日數,存貨週轉率(次),...,不動產、廠房及設備週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),稅前純益佔實收資本比率(%),純益率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,45.66,194.10,124.70,110.88,12.42,5.15,70.87,9.85,...,1.09,0.42,5.92,10.24,50.39,13.02,2.72,31.21,103.54,6.27
1,1102,亞泥,50.31,246.85,105.46,92.77,7.27,4.15,87.95,7.95,...,1.19,0.32,4.79,8.04,31.19,12.04,2.21,23.86,83.19,5.18
2,1103,嘉泥,35.58,832.12,412.74,367.86,881.41,6.30,57.93,2.56,...,1.15,0.13,4.57,6.80,17.23,30.22,1.97,0.00,45.45,0.00
3,1104,環泥,18.08,248.67,154.73,133.99,169.33,4.21,86.69,14.26,...,0.90,0.30,6.68,8.16,19.02,21.21,1.84,23.35,79.11,-0.27
4,1108,幸福,48.70,260.62,250.11,78.18,4.05,4.55,80.21,1.17,...,2.00,0.57,2.54,3.78,5.23,3.34,0.44,27.79,192.58,3.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,9944,新麗,43.16,151.17,139.50,104.37,6.83,5.78,63.14,6.17,...,1.96,0.82,3.27,4.77,17.25,3.20,1.05,13.35,89.23,4.71
990,9945,潤泰新,42.36,1866.63,138.19,37.64,70.08,8.44,43.24,0.64,...,4.45,0.21,44.28,81.83,264.98,206.84,29.03,0.00,0.00,0.00
991,9946,三發地產,62.99,3201.78,170.96,19.97,1560.21,228.84,1.59,0.42,...,38.43,0.55,13.74,41.31,75.11,24.78,6.71,11.07,-62.79,4.64
992,9955,佳龍,31.11,177.34,328.33,97.01,-7.12,25.34,14.40,3.02,...,3.30,1.31,-1.69,-2.69,-7.13,-1.46,-0.61,46.19,24.25,6.36


In [32]:
def revr(data, nmbr):
    cols = data.columns.tolist()
    cols = cols[nmbr:] + cols[:nmbr]
    data = data[cols]
    return data

In [33]:
revr(df,-1)

,現金再投資比率(%),公司代號,公司簡稱,負債佔資產比率(%),長期資金佔不動產、廠房及設備比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),平均收現日數,...,平均銷貨日數,不動產、廠房及設備週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),稅前純益佔實收資本比率(%),純益率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%)
0,6.27,1101,台泥,45.66,194.10,124.70,110.88,12.42,5.15,70.87,...,37.05,1.09,0.42,5.92,10.24,50.39,13.02,2.72,31.21,103.54
1,5.18,1102,亞泥,50.31,246.85,105.46,92.77,7.27,4.15,87.95,...,45.91,1.19,0.32,4.79,8.04,31.19,12.04,2.21,23.86,83.19
2,0.00,1103,嘉泥,35.58,832.12,412.74,367.86,881.41,6.30,57.93,...,142.57,1.15,0.13,4.57,6.80,17.23,30.22,1.97,0.00,45.45
3,-0.27,1104,環泥,18.08,248.67,154.73,133.99,169.33,4.21,86.69,...,25.59,0.90,0.30,6.68,8.16,19.02,21.21,1.84,23.35,79.11
4,3.46,1108,幸福,48.70,260.62,250.11,78.18,4.05,4.55,80.21,...,311.96,2.00,0.57,2.54,3.78,5.23,3.34,0.44,27.79,192.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,4.71,9944,新麗,43.16,151.17,139.50,104.37,6.83,5.78,63.14,...,59.15,1.96,0.82,3.27,4.77,17.25,3.20,1.05,13.35,89.23
990,0.00,9945,潤泰新,42.36,1866.63,138.19,37.64,70.08,8.44,43.24,...,570.31,4.45,0.21,44.28,81.83,264.98,206.84,29.03,0.00,0.00
991,4.64,9946,三發地產,62.99,3201.78,170.96,19.97,1560.21,228.84,1.59,...,869.04,38.43,0.55,13.74,41.31,75.11,24.78,6.71,11.07,-62.79
992,6.36,9955,佳龍,31.11,177.34,328.33,97.01,-7.12,25.34,14.40,...,120.86,3.30,1.31,-1.69,-2.69,-7.13,-1.46,-0.61,46.19,24.25


In [30]:
df.reset_index().drop('index', axis=1)

,公司代號,公司簡稱,負債佔資產比率(%),長期資金佔不動產、廠房及設備比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),平均收現日數,存貨週轉率(次),...,不動產、廠房及設備週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),稅前純益佔實收資本比率(%),純益率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,45.66,194.10,124.70,110.88,12.42,5.15,70.87,9.85,...,1.09,0.42,5.92,10.24,50.39,13.02,2.72,31.21,103.54,6.27
1,1102,亞泥,50.31,246.85,105.46,92.77,7.27,4.15,87.95,7.95,...,1.19,0.32,4.79,8.04,31.19,12.04,2.21,23.86,83.19,5.18
2,1103,嘉泥,35.58,832.12,412.74,367.86,881.41,6.30,57.93,2.56,...,1.15,0.13,4.57,6.80,17.23,30.22,1.97,0.00,45.45,0.00
3,1104,環泥,18.08,248.67,154.73,133.99,169.33,4.21,86.69,14.26,...,0.90,0.30,6.68,8.16,19.02,21.21,1.84,23.35,79.11,-0.27
4,1108,幸福,48.70,260.62,250.11,78.18,4.05,4.55,80.21,1.17,...,2.00,0.57,2.54,3.78,5.23,3.34,0.44,27.79,192.58,3.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,9944,新麗,43.16,151.17,139.50,104.37,6.83,5.78,63.14,6.17,...,1.96,0.82,3.27,4.77,17.25,3.20,1.05,13.35,89.23,4.71
990,9945,潤泰新,42.36,1866.63,138.19,37.64,70.08,8.44,43.24,0.64,...,4.45,0.21,44.28,81.83,264.98,206.84,29.03,0.00,0.00,0.00
991,9946,三發地產,62.99,3201.78,170.96,19.97,1560.21,228.84,1.59,0.42,...,38.43,0.55,13.74,41.31,75.11,24.78,6.71,11.07,-62.79,4.64
992,9955,佳龍,31.11,177.34,328.33,97.01,-7.12,25.34,14.40,3.02,...,3.30,1.31,-1.69,-2.69,-7.13,-1.46,-0.61,46.19,24.25,6.36
